In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import MinMaxScaler
from tabulate import tabulate
from typing import Dict, Tuple

In [ ]:
def imprimir_tabla(df: pd.DataFrame) -> None:
    """
    Imprime un DataFrame en formato tabular.

    Args:
    - df (pd.DataFrame): El DataFrame a imprimir.
    """
    print(tabulate(df, headers=df.columns, tablefmt="orgtbl"))

def transformar_variable(df: pd.DataFrame, columna:str) -> pd.Series:
    """
    Transforma una columna del DataFrame en una serie numérica si no lo es.

    Args:
    - df (pd.DataFrame): El DataFrame.
    - columna (str): El nombre de la columna a transformar.

    Returns:
    - pd.Series: La columna transformada.
    """
    if pd.api.types.is_numeric_dtype(df[columna]):
        return df[columna]
    else:
        return pd.Series(range(len(df[columna])))

def regresion_lineal(df: pd.DataFrame, x:str, y: str) -> Dict[str, float]:
    """
    Realiza una regresión lineal y devuelve los coeficientes y métricas.

    Args:
    - df (pd.DataFrame): El DataFrame.
    - x (str): El nombre de la columna independiente.
    - y (str): El nombre de la columna dependiente.

    Returns:
    - Dict[str, float]: Diccionario con los coeficientes y métricas.
    """
    x_transformado = transformar_variable(df, x)
    modelo = sm.OLS(df[y], sm.add_constant(x_transformado)).fit()
    resumen = modelo.summary2().tables[1]
    imprimir_tabla(resumen)
    coef = resumen['Coef.']
    r2 = modelo.rsquared
    r2_ajustado = modelo.rsquared_adj
    intervalo_confianza = resumen[['[0.025', '0.975]']].iloc[1]
    return {
        'pendiente': coef[0],
        'intercepcion': coef.loc['const'],
        'r2': r2,
        'r2_ajustado': r2_ajustado,
        'banda_inferior': intervalo_confianza[0],
        'banda_superior': intervalo_confianza[1]
    }

def graficar_regresion(df: pd.DataFrame, x: str, y: str, pendiente: float,
                        intercepcion: float, r2: float, r2_ajustado: float,
                        banda_inferior: float, banda_superior: float,
                        colores: Tuple[str,str]) -> None:
    """
    Grafica la regresión lineal junto con las bandas de confianza.

    Args:
    - df (pd.DataFrame): El DataFrame.
    - x (str): El nombre de la columna independiente.
    - y (str): El nombre de la columna dependiente.
    - pendiente (float): La pendiente de la regresión.
    - intercepcion (float): La intercepción de la regresión.
    - r2 (float): El coeficiente de determinación.
    - r2_ajustado (float): El coeficiente de determinación ajustado.
    - banda_inferior (float): La banda inferior del intervalo de confianza.
    - banda_superior (float): La banda superior del intervalo de confianza.
    - colores (Tuple[str, str]): Los colores para la línea de regresión y
    las bandas.
    """
    x_transformado = transformar_variable(df, x)
    df.plot(x=x,y=y, kind='scatter')
    plt.plot(df[x], pendiente * x_transformado + intercepcion, color=colores[0])
    plt.fill_between(df[x],
                    pendiente * x_transformado + banda_inferior,
                    pendiente * x_transformado + banda_superior,
                    alpha=0.2,
                    color=colores[1])

def modelo_regresion_lineal(df: pd.DataFrame, x: str, y: str) -> None:
    """
    Realiza una regresión lineal usando sklearn y grafica los resultados.

    Args:
    - df (pd.DataFrame): El DataFrame.
    - x (str): El nombre de la columna independiente.
    - y (str): El nombre de la columna dependiente.
    """
    print("usando sklearn")
    x_transformado = transformar_variable(df, x).values.reshape(-1, 1)
    modelo = LinearRegression()
    modelo.fit(x_transformado, df[y])
    coef, intercepcion = modelo.coef_[0], modelo.intercept_
    print(f"coef: {coef} intercepcion: {intercepcion}", flush=True)

    df.plot(x=x, y=y, kind='scatter', figsize=(27, 18))
    media = df[y].mean()
    print(f"media: {media}", flush=True)
    plt.plot(df[x], [media] * len(df[x]), color='green')
    plt.plot(df[x], coef * x_transformado + intercepcion, color='red')
    plt.xticks(rotation=90)
    plt.savefig(f'img/lr_{y}_{x}.png')
    plt.close()

def modelo_regresion_lineal_ridge(df: pd.DataFrame, x: str, y: str) -> None:
    """
    Realiza una regresión lineal usando sklearn Ridge y grafica los resultados.

    Args:
    - df (pd.DataFrame): El DataFrame.
    - x (str): El nombre de la columna independiente.
    - y (str): El nombre de la columna dependiente.
    """
    print("usando sklearn con Ridge")
    x_transformado = transformar_variable(df, x).values.reshape(-1, 1)
    y_transformado = df[y].values.reshape(-1, 1)

    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()

    x_transformado = scaler_x.fit_transform(x_transformado)
    y_transformado = scaler_y.fit_transform(y_transformado)

    modelo = Ridge(alpha=1)
    modelo.fit(x_transformado, y_transformado)

    coef, intercepcion = modelo.coef_[0][0], modelo.intercept_[0]
    print(f"coef: {coef} intercepcion: {intercepcion}", flush=True)

    # Predicciones
    predicciones_transformadas = modelo.predict(x_transformado)
    predicciones = scaler_y.inverse_transform(predicciones_transformadas)

    df.plot(x=x, y=y, kind='scatter', figsize=(27, 18))
    media = df[y].mean()
    print(f"media: {media}", flush=True)
    plt.plot(df[x], [media] * len(df[x]), color='green')
    plt.plot(df[x], predicciones, color='red')
    plt.xticks(rotation=90)
    plt.savefig(f'img/lr_{y}_{x}.png')
    plt.close()

def modelo_regresion_lineal_lasso(df: pd.DataFrame, x: str, y: str) -> None:
    """
    Realiza una regresión lineal usando sklearn Lasso y grafica los resultados.

    Args:
    - df (pd.DataFrame): El DataFrame.
    - x (str): El nombre de la columna independiente.
    - y (str): El nombre de la columna dependiente.
    """
    print("usando sklearn")
    x_transformado = transformar_variable(df, x).values.reshape(-1, 1)
    modelo = Lasso(alpha=100000000, max_iter=10000)
    modelo.fit(x_transformado, df[y])
    coef, intercepcion = modelo.coef_[0], modelo.intercept_
    print(f"coef: {coef} intercepcion: {intercepcion}", flush=True)

    df.plot(x=x, y=y, kind='scatter', figsize=(27, 18))
    media = df[y].mean()
    print(f"media: {media}", flush=True)
    plt.plot(df[x], [media] * len(df[x]), color='green')
    plt.plot(df[x], coef * x_transformado + intercepcion, color='red')
    plt.xticks(rotation=90)
    plt.savefig(f'img/lr_{y}_{x}.png')
    plt.close()

# Cargar datos
df = pd.read_csv("csv/typed_uanl.csv") # tipo: pd.DataFrame
imprimir_tabla(df.head(50))

# Agrupar y transformar datos
df_por_sueldo = df.groupby("Fecha").aggregate(
    sueldo_mensual=pd.NamedAgg(column="Sueldo Neto", aggfunc="mean")
).reset_index()
df_por_sueldo["sueldo_mensual"] = df_por_sueldo["sueldo_mensual"] ** 10
imprimir_tabla(df_por_sueldo.head(5))

# Realizar regresión lineal
resultados_regresion = regresion_lineal(df_por_sueldo,
                                        "Fecha", "sueldo_mensual")

# Graficar resultados
graficar_regresion(df=df_por_sueldo, x="Fecha", y="sueldo_mensual",
                    colores=('red', 'orange'), **resultados_regresion)

plt.xticks(rotation=90)
plt.savefig('img/lr_sueldo_mensual_Fecha_m.png')
plt.close()

# Realizar regresión lineal usando sklearn y graficar
modelo_regresion_lineal(df_por_sueldo, "Fecha", "sueldo_mensual")
modelo_regresion_lineal_ridge(df_por_sueldo, "Fecha", "sueldo_mensual")
modelo_regresion_lineal_lasso(df_por_sueldo, "Fecha", "sueldo_mensual")